In [ ]:

import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        continue
       # print(filename)
       # continue
       # print(os.path.join(dirname, filename))


In [ ]:
import os
import PIL
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
train_dir = '/kaggle/input/bird-speciestiny/train'

batch_size = 32
img_height = 180
img_width = 180

train_dir = pathlib.Path(train_dir)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
  seed=123,
  image_size=(img_height, img_width)
)


In [ ]:
train_ds.class_names

In [ ]:
valid_dir = '/kaggle/input/bird-speciestiny/valid'

valid_dir = pathlib.Path(valid_dir)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    seed=123,
    image_size=(img_height, img_width)
)

In [ ]:
valid_ds.class_names

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_ds.class_names[labels[i]])
    plt.axis("off")

In [ ]:
image_count = len(list(train_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
birds = list(train_dir.glob('BANDED BROADBILL/*'))
PIL.Image.open(str(birds[0]))

In [ ]:
labels_csv = pd.read_csv('/kaggle/input/bird-speciestiny/bird-species-tiny.csv')
labels_csv = labels_csv.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
print(labels_csv['data set'])

In [ ]:
train_labels = labels_csv[labels_csv['data set']=='train']
train_labels['filepaths']

In [ ]:
valid_labels = labels_csv[labels_csv['data set']=='valid']['data set']
valid_labels.head()

In [ ]:
test_labels =labels_csv[labels_csv['data set']=='test']['data set']
test_labels.head()

In [ ]:
for images, labels in train_ds:
    print(images.shape)
    print(labels.shape)
  

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

In [ ]:
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
    validation_data= valid_ds,
   epochs=epochs,
)